In [1]:
%load_ext autoreload

%autoreload 2

In [7]:
import torch
import models

# 初始化模型和优化器
audio_model = models.Uni_CMAE(img_size=160, audio_length=1024, norm_pix_loss=True, encoder_depth=12, tr_pos=False)
optimizer = torch.optim.Adam(audio_model.parameters(), lr=0.0001)

# 生成随机输入数据
audio = torch.randn(1, 1024, 128)
video = torch.randn(1, 3, 16, 160, 160)

# 前向传播
loss, loss_mae, loss_mae_a, loss_mae_v, loss_c, mask_a, mask_v, c_acc = audio_model(audio, video)

# # 反向传播
# loss.backward()

# # 检查梯度是否计算正确
# for name, param in audio_model.named_parameters():
#     if param.grad is None:
#         print(f"参数 {name} 没有梯度")
#     else:
#         pass
#         #print(f"参数 {name} 的梯度正常")

# # 保存参数的初始值
# initial_params = {name: param.clone() for name, param in audio_model.named_parameters()}

# # 更新参数
# optimizer.step()

# # 检查参数是否更新
# for name, param in audio_model.named_parameters():
#     if not torch.equal(param, initial_params[name]):
#         pass
#         #print(f"参数 {name} 已更新")
#     else:
#         print(f"参数 {name} 未更新")


A Uni-CMAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
img_size (160, 160) patch_size (16, 16) frames 16 t_patch_size 2
Number of Audio Patches: 512, Visual Patches: 800
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 800, 768])
after mask, audio token shape: torch.Size([1, 256, 768]), visual token shape: torch.Size([1, 80, 768])


In [ ]:
import torch
import models
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import dataloader as dataloader

pth_path = '/home/hao/Project/uni-cmae/egs/voxceleb2/exp/testmae02-audioset-uni-cmae-balNone-lr5e-5-epoch25-bs56-normTrue-c0.01-p1.0-tpFalse-mr-unstructured-0.75/models/best_audio_model.pth'

# 初始化模型和优化器
audio_model = models.Uni_CMAE(img_size=160, audio_length=1024, norm_pix_loss=True, encoder_depth=12, tr_pos=False)

mdl_weight = torch.load(pth_path, map_location=torch.device('cpu'))

# 处理键名不匹配问题
new_state_dict = {}
for k, v in mdl_weight.items():
    if k.startswith('module.'):
        k = k[len('module.'):]
    new_state_dict[k] = v

msg = audio_model.load_state_dict(new_state_dict, strict=False)

print('missing_keys:', msg.missing_keys)
print('unexpected_keys:', msg.unexpected_keys)


In [3]:
import torch
import models
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import dataloader as dataloader


val_audio_conf = {
    'num_mel_bins': 128,
    'target_length': 1024,
    'freqm': 0,
    'timem': 0,
    'mixup': 0,
    'dataset': 'audioset',
    'mode': 'test', # finetuen & test 时出错
    'noise': False,
    'im_res': 160,
    'mean': -5.081,  # 添加 norm_mean
    'std': 4.4849    # 添加 norm_std
}
video_conf = {
    'aa_type': "rand-m7-n4-mstd0.5-inc1",
    'pretrain_rand_flip': True,
    'pretrain_rand_erase_prob': 0.25,
    'pretrain_rand_erase_mode': "pixel",
    'pretrain_rand_erase_count': 1,
    'pretrain_rand_erase_split': False,
    'jitter_aspect_relative': [0.75, 1.3333],
    'jitter_scales_relative': [0.5, 1.0],
    'repeat_aug': 1,
    'num_retries': 10,
    'train_jitter_scales': (256, 320),
    'train_crop_size': 160,
    'train_random_horizontal_flip': True,
    'test_num_ensemble_views': 10,
    'test_num_spatial_crops': 3,
    'test_crop_size': 160,
    'sampling_rate': 4,
    'num_frames': 16,
    'target_fps': 30,
    'mean': (0.45, 0.45, 0.45),
    'std': (0.225, 0.225, 0.225),
    'enable_multi_thread_decode': False,
    'inverse_uniform_sampling': False,
    'use_offset_sampling': True
}

val_set = dataloader.AudiosetDataset(
    dataset_json_file='/home/hao/Project/uni-cmae/egs/voxceleb2/test_data.json',
    label_csv='/home/hao/Project/uni-cmae/egs/voxceleb2/class_labels_indices.csv',
    audio_conf=val_audio_conf,
    video_conf=video_conf,
)

# 获取 val_set 中的一个样本
audio, video, label = val_set[12]

print(f'audio shape: {audio.shape}, video shape: {video.shape}, label: {label}')


Dataset has 36237 samples
Using Label Smoothing: 0.0
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
use dataset mean -5.081 and std 4.485 to normalize the input.
not use noise augmentation
number of classes is 1
now in val mode.
now use frame -1 from total 10 frames
now using 160 * 160 image input
jitter_aspect_relative [0.75, 1.3333] jitter_scales_relative [0.5, 1.0]
Constructing DATASET val...
Perform standard augmentation
audio shape: torch.Size([1024, 128]), video shape: torch.Size([1, 3, 16, 224, 224]), label: tensor([1.])
